# Exercise 2: Backtesting with multiple time series

In this notebook we set out an exercise to do backtesting to compare different models for multiple time series. The solutions we show are only one way of answering these questions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data preparation

The dataset we shall use is the Quarterly overnight trips (in thousands) from 1998 Q1 to 2016 Q4 across
Australia. The number of trips is split by `State`, `Region`, and `Purpose`. 

**In this exercise we are going to forecast the total number of trips for each Region (there are 76 regions therefore we will have 76 time series).**

Source: A new tidy data structure to support
exploration and modeling of temporal data, Journal of Computational and
Graphical Statistics, 29:3, 466-478, doi:10.1080/10618600.2019.1695624.
Shape of the dataset: (24320, 5)

In [ ]:
from skforecast.datasets import fetch_dataset

# Load the data
data = fetch_dataset(name="australia_tourism", raw=True)
data.head()

Pre-process the data by performing the following:
1) Convert the `date_time` column to datetime type
2) Create a dataframe with one column per `Region` which gives the total number of Trips for each date.
3) Ensure the index is `date_time` and resampled to quarterly start `QS`


Later we will use LightGBM. It does not support special JSON characters (e.g., `'`)  in the column name. Let's remove these characters from the column names.

In [ ]:
import re

data = data.rename(columns=lambda x: re.sub("[^A-Za-z0-9_]+", "", x))

Check for missing values.

# Exploratory data analysis

Print the number of data points in the time series, the start time, and the end time of the time series.

Plot the time series summed over all regions

Plot a subsample of the time series from different regions.

Create a quarter of the year feature which could help with this.

# Model definition

Import the classes needed for recursive forecasting for multiple time series and window features from `skforecast`.

Import a transformer from sklearn to scale the data.

Import a linear model and LightGBM

Define a function that extracts features from the target variable:
- A rolling mean with window 4
- A rolling standard deviation with window 4
- A rolling mean with window 12
- A rolling standard deviation with window 12

Hint: https://skforecast.org/0.11.0/user_guides/custom-predictors.html

Define one forecaster for the linear model and one forecaster for the LightGBM model  using `skforecast`. Pass the function in the previous cell as an argument. Use all lags of up to 8.

# Backtesting

Import the relevant backtesting objects from `skforecast`.

Assign the name of the target variables and exogenous variables to variables called `target_cols` and `exog_cols`.

Use the backtesting function to do backtesting with:
- Refitting at every step
- An expanding training window
- Use the `mean_absolute_error` as the error metric
- Use the first 13 datapoints (just over 3 years) as initial training size
- Use exogenous features
- Set the forecast horizon to be 4 steps (i.e., 1 year)

Do this twice, once for the linear forecaster and once for the lightgbm forecaster.

Merge the two metrics dataframe into a single dataframe to make it easier to compare the errors between the linear model and the LightGBM model. 

How often did one model outperform the other model? Remember, the lower the error the better.

It looks like Ridge was better for most time series here. Once again, for the lightgbm model to outperform we might need additional feature engineering, hyperparameter tuning, handling of trends, and more data. There are not any features which help group similar time series together - these type of features are normally well utilised by gradient boosted trees. So in this scenario, a simple linear model works better.

Compute the mean absolute error over all the predictions for both the linear model and the LightGBM model. Compare them.

Plot the predictions made during backtesting alongside the actuals for a random subset of time series to get a better understanding of the errors.

Plot the predictions made during backtesting alongside the actuals for the **highest error** subset of time series to get a better understanding of the errors.